# Agenda

1. Pivot tables
2. Stack, unstack, melt
3. Joins
4. `filter` on columns

In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame

In [2]:
df = DataFrame(np.random.randint(0, 1000, [4,5]),
              index=list('abcd'),
              columns=list('vwxyz'))
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466


In [3]:
df.loc['e'] = [1,2,3,4,5]
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466
e,1,2,3,4,5


In [4]:
new_row = {'w':20, 'x':30, 'z':50}
df.loc['f'] = new_row

In [5]:
df

,v,w,x,y,z
a,906.0,704,539,445.0,618
b,692.0,477,543,649.0,594
c,853.0,732,548,193.0,643
d,501.0,406,614,962.0,466
e,1.0,2,3,4.0,5
f,NaN,20,30,NaN,50


In [6]:
df['w'] = [10, 20, 30, 40, 50, 60]
df

,v,w,x,y,z
a,906.0,10,539,445.0,618
b,692.0,20,543,649.0,594
c,853.0,30,548,193.0,643
d,501.0,40,614,962.0,466
e,1.0,50,3,4.0,5
f,NaN,60,30,NaN,50


In [7]:
df['u'] = {'a':100, 'c':300, 'e':500}

In [8]:
df

,v,w,x,y,z,u
a,906.0,10,539,445.0,618,100.0
b,692.0,20,543,649.0,594,NaN
c,853.0,30,548,193.0,643,300.0
d,501.0,40,614,962.0,466,NaN
e,1.0,50,3,4.0,5,500.0
f,NaN,60,30,NaN,50,NaN


# Grouping

If we have a data frame with:
- One categorical column
- One numeric column

We can use `groupby` to calculate an aggregation method once per category, for all numeric rows matching.

In [9]:
df = pd.read_csv('taxi.csv')
df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0    25.570000
1    17.368569
2    18.406306
3    17.994704
4    18.881648
5    17.211269
6    17.401355
Name: total_amount, dtype: float64

In [10]:
# we can do a 2D grouping

df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

# Pivot table

A pivot table is a 2-dimensional grouping!  It's similar to our multi-index, but uses a table

- We need one categorical column -- this will be the index, for the rows
- A second categorical column - this will be for the columns
- A numeric column
- Aggregation method

In [11]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,25.57,16.941386,19.076807,19.002803,20.518657,20.466667,NaN
2,NaN,17.904989,17.855770,17.359076,17.927913,17.192379,17.401355


In [12]:
df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

In [13]:
df.pivot(index='VendorID', columns='passenger_count', values='total_amount')

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='mean')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,25.57,16.941386,19.076807,19.002803,20.518657,20.466667,NaN
2,NaN,17.904989,17.855770,17.359076,17.927913,17.192379,17.401355


In [16]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='max')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,36.39,252.35,102.35,73.55,73.84,27.30,NaN
2,NaN,137.59,138.84,74.46,72.92,102.11,83.12


In [17]:
# what if we want both mean + standard deviation?
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc=['mean', 'std'])

mean                                                         \
passenger_count      0          1          2          3          4          5   
VendorID                                                                        
1                25.57  16.941386  19.076807  19.002803  20.518657  20.466667   
2                  NaN  17.904989  17.855770  17.359076  17.927913  17.192379   

                                  std                                   \
passenger_count          6          0          1          2          3   
VendorID                                                                 
1                      NaN  15.301791  15.369459  16.049125  15.389968   
2                17.401355        NaN  15.167490  15.233530  12.540837   

                                                  
passenger_count          4          5          6  
VendorID                                          
1                15.845568   6.751543        NaN  
2                14.630713  14.064202  13.363827